In [10]:
import tweepy

In [11]:
API_KEY = 'rzxyaINpAWSj7NxHPzSqJKKvU'
API_SECRET = 'QGoU7jM5EvcmiGgPRwn38ZxyXN60tSzYzNHGU31NWxNn1lCfhL'
ACCESS_TOKEN = '1305084183035944962-Rx3Q7elGUM13xqsejlsGMxvjrYnlj3'
ACCESS_SECRET = 'ry54wOt9i115xxXoC9mmy8Pq2x56qR4kIjnUKkvPPqCHz'

In [12]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

In [ ]:
# Tweets da timeline do usuário autenticado
tweets = tweepy.Cursor(api.home_timeline)
items = tweets.items(5)
for status in items:
  print(status.user.id, status.user.screen_name, status.text, '\n')

In [ ]:
# Tweets da timeline de um usuário do twitter
tweets = tweepy.Cursor(
    api.user_timeline,
    "sbcbrasil", 
    tweet_mode="extended"
)
for status in tweets.items(5):
  print(status.full_text, '\n')

In [ ]:
# Buscar por tweets contendo algum texto
tweets = tweepy.Cursor(
    api.search,
    #
    #'"texto_aqui"' -> busca de texto exato
    #'texto_aqui OR texto_aqui' -> busca um texto ou outro
    #'#texto_aqui' -> busca hashtag
    #'#texto_aqui -filter:retweets' -> não traz os retweets
    #'#texto_aqui from:' -> traz apenas os tweets do usuario
    q=f'tempo',
    lang='pt',
    #until='2021-11-08', ->até essa data
    #since='2021-11-06', ->desde essa data
    tweet_mode="extended" 
)

for status in tweets.items(5):
  print(status.full_text, '\n####\n')

In [ ]:
print(f"Data do tweet: {status.created_at}")
print(f'ID: {status.id}')
print(f'{status.entities["user_mentions"][0]["screen_name"]}')
print(f'{status.user.url}')

In [ ]:
import json
print(json.dumps(status._json, indent=4))

In [18]:
class MyStreamListener(tweepy.StreamListener):
  c = 0
  def on_status(self, status):
    MyStreamListener.c += 1
    texto = status.text
    x = vectorizer.transform([texto])
    y = dtree.predict(x)
    print(f"Sentimento: {y}", status.created_at, status.user.id, texto)

    if MyStreamListener.c > 10:
      MyStreamListener.c = 0
      return False

  def on_error(status, status_code):
    print('Erro: {repr(status_code)}')

In [19]:
myStreamListener = MyStreamListener(api = tweepy.API(wait_on_rate_limit=True))
myStream = tweepy.Stream(auth=auth, listener = myStreamListener)

In [ ]:
myStream.filter(
   #Capturar publicação por id
   #follow = ['785833176'],
    track = ['github'],
    languages = ['pt']
)

Sentimento: [1] 2021-11-10 23:05:31 910856334798663680 O jogo desenvolvido usando apenas software livre, usando Godot para programação do código do jogo e Gimp para desen… https://t.co/h4LtB8qy4R
Sentimento: [1] 2021-11-10 23:08:52 721400322808233985 @amanda_code Pede pra ele criar um txt e ir subindo no github usando os comandos


In [ ]:
import pandas as pd
avaliacoes = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Web Mining/Kindle/reviews.xlsx')
avaliacoes.head()

In [ ]:
#criar vetor de caracteristicas
from sklearn.feature_extraction.text import  TfidfVectorizer
import nltk
nltk.download('stopwords')

In [25]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords = stopwords.remove('não')
vectorizer = TfidfVectorizer(stop_words=nltk.corpus.stopwords.words('portuguese'))

In [26]:
x = vectorizer.fit_transform(avaliacoes.texto)
y = avaliacoes.categoria

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=88)

In [28]:
from sklearn import tree
dtree = tree.DecisionTreeClassifier(random_state=88)
dtree.fit(x_train, y_train)

y_predict = dtree.predict(x_test)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.8662175168431184